In [1]:
import base64

try:
    from io import StringIO
except ImportError:
    from StringIO import StringIO

import pandas

from traitlets import Unicode, Instance, observe
from ipywidgets import DOMWidget
from IPython import display

In [2]:
%%html
<style>
    .uploader-widget{
        border: dashed 2px grey;
        opacity: 0.5;
        background-color: #efefef;
        color: grey;
        position: relative;
    }
    .uploader-widget.uploader-dragged {
        background-color: #333;
        color: white;
    }
    .uploader-widget input{
        opacity: 0;
        width: 100%;
        position: absolute;
        width: 100%;
        height: 100%;
        top: 0;
    }
    .uploader-widget h3 {
        text-align: center;
        margin: 0;
        padding: 20px;
    }
</style>

In [3]:
%%javascript
require.undef("widget-uploader")
define(
"widget-uploader",
[
    "underscore",
    "jquery",
    "nbextensions/widgets/widgets/js/widget"
],
function(_, $, widget){
    var UploaderView = widget.DOMWidgetView.extend({
        className: "uploader-widget",
        events: {
            dragenter: "onDragEnter",
            dragleave: "onDragLeave",
            drop: "onDrop",
            "change input": "onChangeInput"
        },
        
        render: function(){
            var that = this;

            this.$label = $("<h3/>").appendTo(this.$el);

            this.$fileField = $("<input/>", {type: "file"})
                .appendTo(this.$el);

            this.update();
        },
        onChangeInput: function(){
            this.setFile(this.$fileField[0].files[0]);
        },
        setFile: function(file){
            var reader  = new FileReader();

            reader.addEventListener("load", _.bind(function(){
                this.model.set("base64_data", reader.result);
                this.touch();
            }, this), false);

            reader.readAsDataURL(file);
        },
        update: function() {
            this.$label.text(this.model.get("label"));
        },
                                                   
        onDragEnter: function(evt){ this.$el.addClass("uploader-dragged"); console.log(evt);},
        onDragLeave: function(evt){ this.$el.removeClass("uploader-dragged"); console.log(evt);},
        onDrop: function(evt){
            evt.preventDefault();
            evt.stopImmediatePropagation();
            this.$el.removeClass("uploader-dragged");
            console.log(evt);
            this.setFile(evt.originalEvent.dataTransfer.files[0]);
        }
    });
    return {
        UploaderView: UploaderView
    }
});

<IPython.core.display.Javascript object>

In [4]:
class BaseUploaderWidget(DOMWidget):
    _view_module = Unicode("widget-uploader").tag(sync=True)
    _view_name = Unicode("UploaderView").tag(sync=True)
    base64_data = Unicode().tag(sync=True)
    label = Unicode("Upload a File").tag(sync=True)
    
    @observe("base64_data")
    def _update_label(self, change):
        self.label = self._label() or self.label
    
    def _label(self):
        return "{} bytes uploaded {}".format(
            len(self.base64_data),
            self.base64_data.split(";")[0])

uploader = BaseUploaderWidget()
uploader

In [5]:
class CSVUploader(BaseUploaderWidget):
    dataframe = Instance(klass=pandas.DataFrame, allow_none=True)
    label = Unicode("Upload a CSV File").tag(sync=True)
    
    def _label(self):
        self.label = "{} rows by {} columns".format(
            len(self.dataframe),
            len(self.dataframe.columns),
        )

    @observe("base64_data")
    def _update_dataframe(self, new):
        csv = StringIO()
        csv.write(u"{}".format(base64.decodestring(self.base64_data.split(",")[1])))
        csv.seek(0)
        self.dataframe = pandas.read_csv(csv)

In [6]:
csv = CSVUploader()
csv

In [7]:
display.display_html(csv)

In [8]:
uploader

In [9]:
%%html
<a href="http://example.com">link</a>

In [6]:
BaseUploaderWidget()

NameError: name 'BaseUploaderWidget' is not defined

In [4]:
import base64 # You need it if you define binary uploads
from __future__ import print_function # py 2.7 compat.
import ipywidgets as widgets # Widget definitions.
from traitlets import List, Unicode  # Traitlets needed to add synced attributes to the widget.

class FileWidget(widgets.DOMWidget):
    _view_name = Unicode('FilePickerView').tag(sync=True)
    _view_module = Unicode('filepicker').tag(sync=True)
    filenames = List([]).tag( sync=True)
    # values = List(trait=Unicode, sync=True)

    def __init__(self, **kwargs):
        """Constructor"""
        super().__init__(**kwargs)

        # Allow the user to register error callbacks with the following signatures:
        #    callback()
        #    callback(sender)
        self.errors = widgets.CallbackDispatcher(accepted_nargs=[0, 1])

        # Listen for custom msgs
        self.on_msg(self._handle_custom_msg)

    def _handle_custom_msg(self, content):
        """Handle a msg from the front-end.

        Parameters
        ----------
        content: dict
            Content of the msg."""
        if 'event' in content and content['event'] == 'error':
            self.errors()
            self.errors(self)

In [5]:
%%javascript
requirejs.undef('filepicker');

define('filepicker', ["jupyter-js-widgets"], function(widgets) {

    var FilePickerView = widgets.DOMWidgetView.extend({
        render: function(){
            // Render the view using HTML5 multiple file input support.
            this.setElement($('<input class="fileinput" multiple="multiple" name="datafile"  />')
                .attr('type', 'file'));
        },

        events: {
            // List of events and their handlers.
            'change': 'handle_file_change',
        },

        handle_file_change: function(evt) { 
            // Handle when the user has changed the file.

            // Save context (or namespace or whatever this is)
            var that = this;

            // Retrieve the FileList object
            var files = evt.originalEvent.target.files;
            var filenames = [];
            var file_readers = [];
            console.log("Reading files:");

            for (var i = 0; i < files.length; i++) {
                var file = files[i];
                console.log("Filename: " + file.name);
                console.log("Type: " + file.type);
                console.log("Size: " + file.size + " bytes");
                filenames.push(file.name);

                // Read the file's textual content and set value_i to those contents.
                file_readers.push(new FileReader());
                file_readers[i].onload = (function(file, i) {
                    return function(e) {
                        that.model.set('value_' + i, e.target.result);
                        that.touch();
                        console.log("file_" + i + " loaded: " + file.name);
                    };
                })(file, i);

                file_readers[i].readAsText(file);
            }

            // Set the filenames of the files.
            this.model.set('filenames', filenames);
            this.touch();
        },
    });

    // Register the FilePickerView with the widget manager.
    return {
        FilePickerView: FilePickerView
    };
});

<IPython.core.display.Javascript object>

In [7]:
file_widget = FileWidget()

def file_loaded(change):
    '''Register an event to save contents when a file has been uploaded.'''
    print(change['new'])
    i = int(change['name'].split('_')[1])
    fname = file_widget.filenames[i]
    print('file_loaded: {}'.format(fname))

def file_loading(change):
    '''Update self.model when user requests a list of files to be uploaded'''
    print(change['new'])
    num = len(change['new'])
    traits = [('value_{}'.format(i), Unicode(sync=True)) for i in range(num)]
    file_widget.add_traits(**dict(traits))
    for i in range(num):
        file_widget.observe(file_loaded, 'value_{}'.format(i))
file_widget.observe(file_loading, names='filenames')

def file_failed():
    print("Could not load some file contents.")
file_widget.errors.register_callback(file_failed)


file_widget

['Firefox @ automag.desktop', 'COMSOL.desktop', '.directory']
[Desktop Entry]
Comment[en_GB]=
Comment=
Comment[de]=
Exec=ssh -X -l guest automag firefox -P default2
GenericName[en_GB]=firefox @ automag
GenericName=firefox @ automag
GenericName[de]=firefox @ automag
Icon=/usr/share/pixmaps/firefox.png
MimeType=
Name=Firefox @ automag
Name[de]=Firefox @ automag
Path=
StartupNotify=true
Terminal=false
TerminalOptions=
Type=Application
X-DBUS-ServiceName=
X-DBUS-StartupType=
X-DCOP-ServiceType=
X-KDE-SubstituteUID=false
X-KDE-Username=

file_loaded: Firefox @ automag.desktop
[Desktop Entry]
Comment[en_GB]=
Comment=
Exec=ssh -X holodeck '/opt/comsol/bin/comsol'
GenericName[en_GB]=
GenericName=
Icon=bball
MimeType=
Name[en_GB]=COMSOL
Name=COMSOL
Path=
StartupNotify=true
Terminal=false
TerminalOptions=
Type=Application
X-DBUS-ServiceName=
X-DBUS-StartupType=
X-KDE-SubstituteUID=false
X-KDE-Username=

file_loaded: COMSOL.desktop
[Desktop Entry]
BgImage=
Icon=user-desktop
Name=Desktop
Name[af]=